# Daily Filtering

After 5.30pm

In [173]:
from nselib import capital_market
import concurrent.futures
from tqdm import tqdm
import warnings
import numpy as np
import pandas as pd
from datetime import datetime
import re
from nse import nse_self
nse = nse_self()
warnings.filterwarnings("ignore")

In [174]:
today = '22-08-2025'

In [175]:
today_stocks_info = nse.get_nse_equities_by_date(today)
stocks = today_stocks_info[today_stocks_info['SERIES'] == 'EQ']['SYMBOL'].tolist()

In [176]:
today_stocks_info.loc[
    today_stocks_info['SYMBOL'] == 'BODALCHEM', ['SYMBOL', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'CLOSE_PRICE','TTL_TRD_QNTY']
    ].rename(columns={
        'OPEN_PRICE': 'O',
        'HIGH_PRICE':'H',
        'LOW_PRICE': 'L',
        'CLOSE_PRICE': 'C',
        'TTL_TRD_QNTY': 'Volume'
    })

,SYMBOL,O,H,L,C,Volume
447,BODALCHEM,67.78,68.09,66.6,66.76,312283


In [177]:
nse.get_nse_stock_by_duration('BODALCHEM', '1W')

,Symbol,Series,Date,PrevClose,OpenPrice,HighPrice,LowPrice,LastPrice,ClosePrice,AveragePrice,TotalTradedQuantity,TurnoverInRs,No.ofTrades,DeliverableQty,%DlyQttoTradedQty,"ï»¿""Symbol"""
0,NaN,EQ,18-Aug-2025,65.85,66.90,68.22,66.04,67.00,67.31,67.24,421176,28321914.98,4426,228414,54.23,BODALCHEM
1,NaN,EQ,19-Aug-2025,67.31,67.69,68.52,67.17,68.25,68.09,68.02,228489,15542167.64,2090,115966,50.75,BODALCHEM
2,NaN,EQ,20-Aug-2025,68.09,68.20,69.20,67.52,68.36,68.49,68.54,292704,20062703.76,2917,143022,48.86,BODALCHEM
3,NaN,EQ,21-Aug-2025,68.49,68.06,68.80,67.20,67.63,67.47,67.98,154825,10524502.55,1280,101440,65.52,BODALCHEM
4,NaN,EQ,22-Aug-2025,67.47,67.78,68.09,66.60,66.90,66.76,67.24,312283,20997412.05,3100,166474,53.31,BODALCHEM


# level 1 - MA 200 & Vol 2.25x

In [178]:
filtered_stocks_1 = []

def process_stock(stock):
    try:
        data = nse.get_nse_stock_by_duration(symbol=stock, period='1Y')
    except:
        # skipping; stock with trades less than 1000 TDs
        return None


    # -- MA 200 -- 
    MA = 200
    close_series = data[::-1]['ClosePrice'][:MA]
    if close_series.empty or len(close_series.dropna()) == 0:
        return None 
    
    close = close_series.replace({',': ''}, regex=True).astype(float)
    if len(close) < MA:
        return None

    closePrice = close.iloc[0]
    MA_200 = close.mean()

    # TODO: considering adding some buffer
    if closePrice < MA_200:
        # NOTE: skipping; stock has been listed for less than 200 days
        return None


    # -- Volume > 2.25x 10D Avg --
    try:
        volume_data = capital_market.deliverable_position_data(stock, period='1M')['TradedQty']
        volume = volume_data[::-1].str.replace(',', '', regex=False).astype(int)
    except:
        return None

    current_vol = volume.iloc[0]
    # volume needs to above 100k
    if current_vol > 100_000:
        last_10_days_avg = volume.iloc[1:11].mean()
        if current_vol > last_10_days_avg * 2.25:
            return stock

    return None

print(f'Total no. of stocks: {len(stocks)}')
print()

with concurrent.futures.ThreadPoolExecutor(max_workers=11) as executor:
    futures = [executor.submit(process_stock, stock) for stock in stocks]
    for f in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc='Flitering'):
        result = f.result()
        if result:
            filtered_stocks_1.append(result)

print(f'Filtered stocks: {len(filtered_stocks_1)}')

Total no. of stocks: 2099



Flitering: 100%|██████████| 2099/2099 [10:13<00:00,  3.42it/s]

Filtered stocks: 43


# level 2 - excluding ETFs

In [179]:
exclusions = ["ETF", "GOLD", "SILVER", "BEES", "NIFTY", 'MAFANG', 'MAHKTECH']
excluded_stocks = []
filtered_stocks_2 = []

for stk in filtered_stocks_1:
    if any(excl in stk for excl in exclusions):
        excluded_stocks.append(stk)
    else:
        filtered_stocks_2.append(stk)

print(f"Excluded stocks: {excluded_stocks}")
len(filtered_stocks_2)

Excluded stocks: ['BANKIETF', 'BSE500IETF', 'HDFCNIFTY', 'LTGILTBEES']


39

# level 3 - higher opens & higher closes

In [180]:
filtered_stocks_3 = []

def check_price_movement(stock):
    try:
        # get lastest 7 days of data
        data = nse.get_nse_stock_by_duration(symbol=stock, period='1W')
        # latest
        data = data[::-1]  
        close_prices = data['ClosePrice'].replace({',': ''}, regex=True).astype(float)
        open_prices = data['OpenPrice'].replace({',': ''}, regex=True).astype(float)

        if len(close_prices) >= 2 and len(open_prices) >= 1:
            today_close = close_prices.iloc[0]
            prev_close = close_prices.iloc[1]
            today_open = open_prices.iloc[0]
            prev_open = open_prices.iloc[1]

            # Condition: today's close > previous close AND today's close > open
            if today_close > prev_close and today_close > today_open and today_open > prev_open:
                if stock == 'BODALCHEM':
                    print(today_close)
                    print(prev_close)
                return stock
    except Exception as e:
        return None
    return None


with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    futures = [executor.submit(check_price_movement, stock) for stock in filtered_stocks_2]
    for f in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc='Price Check'):
        result = f.result()
        if result:
            filtered_stocks_3.append(result)

# sorting alphabetically
filtered_stocks_3.sort()
print(f"Filtered stocks: {len(filtered_stocks_3)}")
print(filtered_stocks_3)

Price Check: 100%|██████████| 39/39 [00:02<00:00, 17.60it/s]

Filtered stocks: 19
['AIROLAM', 'APOLLO', 'AVANTEL', 'BALKRISHNA', 'BOMDYEING', 'FEDFINA', 'GOCLCORP', 'GPIL', 'GUJAPOLLO', 'HERANBA', 'HUBTOWN', 'IZMO', 'KOLTEPATIL', 'LEMONTREE', 'LOTUSEYE', 'OBCL', 'ROHLTD', 'SILGO', 'TOUCHWOOD']


# level 4 - previous 10 days's daily vol > 100k

In [181]:
filtered_stocks_4 = []

def check_volume_consistency(stock):
    try:
        # Get last 15 days of data (to ensure we have at least 10 trading days)
        volume_data = nse.get_nse_stock_by_duration(symbol=stock, period='1M')['TotalTradedQuantity']
        volume = volume_data[::-1]

        # Take last 10 trading days
        last_10_vols = volume.iloc[:10]

        # Check if all volumes > 100k
        if len(last_10_vols) < 10:
            return None

        if all(v > 100_000 for v in last_10_vols):
            return stock
    except:
        return None
    return None


with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    finals = [executor.submit(check_volume_consistency, stock) for stock in filtered_stocks_3]
    for f in tqdm(concurrent.futures.as_completed(finals), total=len(finals), desc='Volume Consistency'):
        result = f.result()
        if result:
            filtered_stocks_4.append(result)

# Sort alphabetically
filtered_stocks_4.sort()
print(f"Filtered Stocks_4: {len(filtered_stocks_4)}")
print(filtered_stocks_4)

Volume Consistency: 100%|██████████| 19/19 [00:01<00:00, 12.85it/s]

Filtered Stocks_4: 7
['APOLLO', 'AVANTEL', 'BOMDYEING', 'FEDFINA', 'GPIL', 'HUBTOWN', 'LEMONTREE']


# final

In [182]:
filtered_stocks_4.sort()
np.array(filtered_stocks_4)

array(['APOLLO', 'AVANTEL', 'BOMDYEING', 'FEDFINA', 'GPIL', 'HUBTOWN',
       'LEMONTREE'], dtype='<U9')

# prompt & pat bateman score

In [183]:
def generate_prompt(stk_symbol, today_stocks_info):
    
    ohlc = today_stocks_info.loc[today_stocks_info['SYMBOL'] == stk_symbol,
                                ['SYMBOL', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'CLOSE_PRICE']
    ].values.tolist()[0][1:]

    hist_data = nse.get_nse_stock_by_duration(symbol=stk_symbol, period='1Y')[::-1]
    closes = hist_data['ClosePrice'].replace({',': ''}, regex=True).astype(float)
    highs = hist_data['HighPrice'].replace({',': ''}, regex=True).astype(float)

    ma20 = closes.iloc[:20].mean()
    ma50 = closes.iloc[:50].mean()
    ma200 = closes.iloc[:200].mean()
    today_close = closes.iloc[0]
    ma20_gap = round(((today_close - ma20) / ma20) * 100, 2)
    ma50_gap = round(((today_close - ma50) / ma50) * 100, 2)
    ma200_gap = round(((today_close - ma200) / ma200) * 100, 2)

    yesterday_close = closes.iloc[1]
    today_close = closes.iloc[0]
    change_pct = ((today_close - yesterday_close) / yesterday_close) * 100

    # TODO: handle stock splits
    high_52 = highs.max()
    high_gap = round(((high_52 - today_close) / high_52) * 100, 2)

    # TODO: candle height? upper tick!
    prompt = f"""
        I'm evaluating **NSE:{stk_symbol}** for a potential **swing buy position**.  
        Please assess this setup based on my strategy context and your expert technical insights.
        Kindly provide a **combined score (1 to 10)** reflecting your preference for a buy at this point also considering the below details.
        

        Stock Details:  
        - Market closed today's 1D chart with:  
        **Open**: {ohlc[0]} | **High**: {ohlc[1]} | **Low**: {ohlc[2]} | **Close**: {ohlc[3]} | **Change**: {change_pct:+.2f}%
        - **20 MA**: {ma20:.2f} (Gap: {ma20_gap:+.2f}%)  
        - **50 MA**: {ma50:.2f} (Gap: {ma50_gap:+.2f}%)  
        - **200 MA**: {ma200:.2f} (Gap: {ma200_gap:+.2f}%)  
        - **RSI**: to_be_filled
        - **MACD** to_be_filled Signal 
        - **ADX** to_be_filled
        - 52-week High: {high_52:.2f} | Gap: {high_gap:+.2f}%
        - Volume is **2.25x or more** above the average of the last 10 trading days.
        - Both **today's** and **yesterday's** candles show **higher opens and higher closes**.


        Additional IMP Info (Please fill):
        - Earnings in (days):
        - If already declared then earnings Recap (Date, Miss/Meet/Beat, Guidance sentiment - Strong/Neutral/Weak): 
        - Sector outlook:  
        - Sector strength: Good / Bad  
        - News related to this stock: Good/Bad, any global news (like tariff) and check for any 
          sector-specific news or analyst upgrades/downgrades that might influence sector or stock movements
        - Institutional investor interest:  
        - Analysts' research:  

        ---

        📍 **Ideal Entry Zone** (Now or wait):  
        🎯 **Final Verdict (Score out of 10)**:  
        """
    
    pat_bateman = f"pat_bateman_score(fin, shareholding, research, {ma50_gap:+.2f}, {ma20_gap:+.2f}, {high_gap:+.2f}, rsi, (macd, signal), adx, news)"

    return prompt, pat_bateman
    

TODO: RSI & MACD

In [184]:
def pat_bateman_score(
        financials_score, shareholding_score, analyst_rating_score,  # 0 to 1
        price_vs_50ma_pct, price_vs_20ma_pct,                        # gap percentages
        gap_52wk_H, rsi, macd, adx, news_positive                    # gap percentage, 
):
    
    # weightage (make sure all sum up to 10)
    weights = {
        'financials':      1.075,           
        'shareholding':    1.05,
        'analyst':         1.05,             # 3.2

        '50ma':            1.05,
        '20ma':            1.05,            # 2.1

        '52wk_high':       0.875,
        'rsi':             0.975,
        'macd':            0.975,
        'adx':             0.8,
        'news':            1.1              # 4.7
    }

    def norm_50ma(pct):
        # full score if within 5-12%; linearly drop outside
        if 5 <= pct <= 12:
            return 1.0
        # outside range: decrease linearly to 0 at 0% or 20%
        if pct < 5:
            return max(0.0, pct / 5)
        return max(0.0, (20 - pct) / 8)

    def norm_20ma(pct):
        if 2 <= pct <= 6:
            return 1.0
        if pct < 2:
            return max(0.0, pct / 2)
        return max(0.0, (15 - pct) / 9)

    def norm_52wk(pct):
        if 8 <= pct <= 30:
            return 0.9
        if 0 <= pct < 8: # TODO: add offset of 0.35 because sometimes at 52 week high breakout may happen
            return min(0.9, pct / 8 + 0.35)
        if 30 < pct <= 60:
            return (60 - pct) / 30
        return 0.0

    def norm_range(value, low, high):
        # perfect score if within [low, high]
        if low <= value <= high:
            return 1.0
        # outside: linear drop to 0 at bounds*2
        if value < low:
            return max(0.0, value / low)
        return max(0.0, (high*2 - value) / high)
    
    # TODO: test cases 
    def norm_macd(macd, signal, ideal_stretch_pct=2.0, max_stretch_pct=5.0, neg_stretch_pct=1.25):
        diff = macd - signal
        stretch_pct = (diff / abs(signal)) * 100  # percentage difference

        if macd > 0 and signal > 0:
            zone = 'bullish'
        elif macd < 0 and signal < 0:
            zone = 'bearish'
        else:
            zone = 'crossover'

        if zone == 'crossover' and abs(stretch_pct) > max_stretch_pct:
            return 0.0

        if zone == "bullish":
            # MACD >= Signal
            if diff >= 0: 
                if stretch_pct <= ideal_stretch_pct:
                    return 1.0
                elif stretch_pct < max_stretch_pct:
                    score = 1.0 - (stretch_pct - ideal_stretch_pct) / (max_stretch_pct - ideal_stretch_pct)
                    return max(score, 0.5)
                else:
                    return 0.45  # stretched but still bullish
            # MACD < Signal --> weakening
            else:  
                if -neg_stretch_pct < stretch_pct < 0:
                    return 0.45 - ((abs(stretch_pct) / neg_stretch_pct) * (0.45 - 0.25))
                else:
                    return 0.23  # weak bullish
                
        if zone == "bearish":
            if diff <= 0:  # MACD < Signal
                if abs(stretch_pct) >= ideal_stretch_pct:
                    return 0.15
                elif abs(stretch_pct) < max_stretch_pct:
                    return 0.1
                else:
                    return 0.0
            else:  # MACD > Signal → weakening bearish
                return 0.3  # slightly less bearish

        # Crossover Zone
        return 0.25  # neutral/indecisive


    # Compute each normalized factor
    scores = {}
    # TODO: 
    scores['financials']   = financials_score*weights['financials']
    scores['shareholding'] = shareholding_score*weights['shareholding']
    scores['analyst']      = analyst_rating_score*weights['analyst']

    scores['50ma']         = norm_50ma(price_vs_50ma_pct)
    scores['20ma']         = norm_20ma(price_vs_20ma_pct)

    scores['52wk_high']    = norm_52wk(gap_52wk_H)
    scores['rsi']          = norm_range(rsi, 48, 66)
    scores['macd']         = norm_macd(macd[0], macd[1])
    scores['adx']          = norm_range(adx, 20, 60)
    scores['news']         = news_positive*weights['news']

    raw_score = sum(scores[k] * w for k, w in weights.items())

    return round(raw_score, 2)

# generating prompt & pat bateman score

In [191]:
stk_symbol = filtered_stocks_4[-1]
print(stk_symbol)

LEMONTREE


In [192]:
# stk_symbol = 'MOLDTKPAC'
prompt, pat_bateman = generate_prompt(stk_symbol, today_stocks_info)
print(prompt)


        I'm evaluating **NSE:LEMONTREE** for a potential **swing buy position**.  
        Please assess this setup based on my strategy context and your expert technical insights.
        Kindly provide a **combined score (1 to 10)** reflecting your preference for a buy at this point also considering the below details.
        

        Stock Details:  
        - Market closed today's 1D chart with:  
        **Open**: 167.0 | **High**: 174.9 | **Low**: 166.0 | **Close**: 169.45 | **Change**: +1.87%
        - **20 MA**: 149.83 (Gap: +13.10%)  
        - **50 MA**: 146.69 (Gap: +15.51%)  
        - **200 MA**: 139.45 (Gap: +21.51%)  
        - **RSI**: to_be_filled
        - **MACD** to_be_filled Signal 
        - **ADX** to_be_filled
        - 52-week High: 174.90 | Gap: +3.12%
        - Volume is **2.25x or more** above the average of the last 10 trading days.
        - Both **today's** and **yesterday's** candles show **higher opens and higher closes**.


        Additional IMP Inf

In [141]:
pat_bateman

'pat_bateman_score(fin, shareholding, research, +26.90, +9.00, +1.11, rsi, (macd, signal), adx, news)'

if there is no analyts' rating then 0.6

In [142]:
pat_bateman_score(0.85, 0.85, 0.75, +26.90, +9.00, +1.11, 69.91, (42.04, 45.98), 36.11, 0.65)

6.6

In [636]:

    # TODO: test cases 
    def norm_macd(macd, signal, ideal_stretch_pct=2.0, max_stretch_pct=5.0, neg_stretch_pct=1.25):
        diff = macd - signal
        stretch_pct = (diff / abs(signal)) * 100  # percentage difference

        if macd > 0 and signal > 0:
            zone = 'bullish'
        elif macd < 0 and signal < 0:
            zone = 'bearish'
        else:
            zone = 'crossover'

        if zone == 'crossover' and abs(stretch_pct) > max_stretch_pct:
            return 0.0

        if zone == "bullish":
            # MACD >= Signal
            if diff >= 0: 
                if stretch_pct <= ideal_stretch_pct:
                    return 1.0
                elif stretch_pct < max_stretch_pct:
                    score = 1.0 - (stretch_pct - ideal_stretch_pct) / (max_stretch_pct - ideal_stretch_pct)
                    return max(score, 0.5)
                else:
                    return 0.45  # stretched but still bullish
            # MACD < Signal --> weakening
            else:  
                if -neg_stretch_pct < stretch_pct < 0:
                    return 0.45 - ((abs(stretch_pct) / neg_stretch_pct) * (0.45 - 0.25))
                else:
                    return 0.23  # weak bullish
                
        if zone == "bearish":
            if diff <= 0:  # MACD < Signal
                if abs(stretch_pct) >= ideal_stretch_pct:
                    return 0.15
                elif abs(stretch_pct) < max_stretch_pct:
                    return 0.1
                else:
                    return 0.0
            else:  # MACD > Signal → weakening bearish
                return 0.3  # slightly less bearish

        # Crossover Zone
        return 0.25  # neutral/indecisive

# WIP (to get 15-min)

In [590]:
import yfinance as yf

ticker = yf.Ticker("SBIN.NS")
data = ticker.history(interval="15m", period="1d")
print(data[['Open', 'High', 'Low', 'Close', 'Volume']])


DEBUG:yfinance:Entering history()
DEBUG:peewee:('SELECT "t1"."key", "t1"."value" FROM "_kv" AS "t1" WHERE ("t1"."key" = ?) LIMIT ? OFFSET ?', ['SBIN.NS', 1, 0])
DEBUG:yfinance: Entering history()
DEBUG:yfinance:SBIN.NS: Yahoo GET parameters: {'range': '1d', 'interval': '15m', 'includePrePost': False, 'events': 'div,splits,capitalGains'}
DEBUG:yfinance:  Entering get()
DEBUG:yfinance:   Entering _make_request()
DEBUG:yfinance:url=https://query2.finance.yahoo.com/v8/finance/chart/SBIN.NS
DEBUG:yfinance:params={'range': '1d', 'interval': '15m', 'includePrePost': False, 'events': 'div,splits,capitalGains'}
DEBUG:yfinance:    Entering _get_cookie_and_crumb()
DEBUG:yfinance:cookie_mode = 'basic'
DEBUG:yfinance:     Entering _get_cookie_and_crumb_basic()
DEBUG:yfinance:reusing cookie
DEBUG:yfinance:reusing crumb
DEBUG:yfinance:     Exiting _get_cookie_and_crumb_basic()
DEBUG:yfinance:    Exiting _get_cookie_and_crumb()
DEBUG:yfinance:response code=429
DEBUG:yfinance:toggling cookie strategy b

YFRateLimitError: Too Many Requests. Rate limited. Try after a while.